In [33]:
# ! kaggle competitions download -c titanic

In [34]:
# ! unzip titanic.zip

In [35]:
# ! pwd

In [36]:
# ! pip install optuna

In [60]:
import pandas as pd
import seaborn as sns
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np

X_train = pd.read_csv('/opt/notebooks/kaggle/Kaggle-competitions/titanic/train.csv')
X_test = pd.read_csv('/opt/notebooks/kaggle/Kaggle-competitions/titanic/test.csv')

y_train = X_train['Survived']
X_train.drop(columns='Survived', inplace=True)

train_df = pd.concat([X_train, X_test], ignore_index=True)
train_df = train_df.drop(columns=['Name', 'Ticket', 'Cabin'])
train_df['Fare'].fillna(train_df['Fare'].mean(), inplace=True)

In [38]:
train_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,male,22.0,1,0,7.2500,S
1,2,1,female,38.0,1,0,71.2833,C
2,3,3,female,26.0,0,0,7.9250,S
3,4,1,female,35.0,1,0,53.1000,S
4,5,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
1304,1305,3,male,NaN,0,0,8.0500,S
1305,1306,1,female,39.0,0,0,108.9000,C
1306,1307,3,male,38.5,0,0,7.2500,S
1307,1308,3,male,NaN,0,0,8.0500,S


Предскажем пропущенный данные в возрасте

In [39]:
test_age = train_df[train_df['Age'].isna()]
train_age = train_df[train_df['Age'].notna()]

In [40]:
# обучаем модель на данных, где возраст не пропущен
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X = train_age[['Pclass', 'SibSp', 'Parch', 'Fare']]
y = train_age['Age']
X['Fare'].fillna(X['Fare'].mean(), inplace=True)

# X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(X, y, test_size=0.3, random_state=42)
model = RandomForestRegressor(n_estimators=1000, max_depth=10, random_state=42)
model.fit(X, y)
predicted_ages = model.predict(test_age[['Pclass', 'SibSp', 'Parch', 'Fare']]).astype(int)
train_df.loc[train_df['Age'].isna(), 'Age'] = predicted_ages

/tmp/ipykernel_53313/1819819962.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
train_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,3,male,22.0,1,0,7.2500,S
1,2,1,female,38.0,1,0,71.2833,C
2,3,3,female,26.0,0,0,7.9250,S
3,4,1,female,35.0,1,0,53.1000,S
4,5,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
1304,1305,3,male,28.0,0,0,8.0500,S
1305,1306,1,female,39.0,0,0,108.9000,C
1306,1307,3,male,38.5,0,0,7.2500,S
1307,1308,3,male,28.0,0,0,8.0500,S


In [42]:
train_df['solo'] = ((train_df['SibSp'] != 0) | (train_df['Parch'] != 0)).astype(int)

In [43]:
train_df['ageGroup'] = pd.cut(train_df['Age'], bins=[0, 12, 20, 40, 60, 80], labels=False)

In [44]:
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1

In [45]:
train_df['FareGroup'] = pd.cut(train_df['Fare'], bins=[-0.1, 7, 14, 21, 30, 1000], labels=False).astype(int)

In [46]:
train_df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,solo,ageGroup,FamilySize,FareGroup
0,1,3,male,22.0,1,0,7.2500,S,1,2,2,1
1,2,1,female,38.0,1,0,71.2833,C,1,2,2,4
2,3,3,female,26.0,0,0,7.9250,S,0,2,1,1
3,4,1,female,35.0,1,0,53.1000,S,1,2,2,4
4,5,3,male,35.0,0,0,8.0500,S,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,3,male,28.0,0,0,8.0500,S,0,2,1,1
1305,1306,1,female,39.0,0,0,108.9000,C,0,2,1,4
1306,1307,3,male,38.5,0,0,7.2500,S,0,2,1,1
1307,1308,3,male,28.0,0,0,8.0500,S,0,2,1,1


In [47]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Sex          1309 non-null   object 
 3   Age          1309 non-null   float64
 4   SibSp        1309 non-null   int64  
 5   Parch        1309 non-null   int64  
 6   Fare         1309 non-null   float64
 7   Embarked     1307 non-null   object 
 8   solo         1309 non-null   int64  
 9   ageGroup     1309 non-null   int64  
 10  FamilySize   1309 non-null   int64  
 11  FareGroup    1309 non-null   int64  
dtypes: float64(2), int64(8), object(2)
memory usage: 122.8+ KB


In [48]:
train_df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,solo,ageGroup,FamilySize,FareGroup
count,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000
mean,655.000000,2.294882,29.749939,0.498854,0.385027,33.295479,0.396486,1.942704,1.883881,2.148969
std,378.020061,0.837836,13.363519,1.041658,0.865560,51.738879,0.489354,0.840809,1.583639,1.321994
min,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,328.000000,2.000000,22.000000,0.000000,0.000000,7.895800,0.000000,2.000000,1.000000,1.000000
50%,655.000000,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000,2.000000,1.000000,2.000000
75%,982.000000,3.000000,37.000000,1.000000,0.000000,31.275000,1.000000,2.000000,2.000000,4.000000
max,1309.000000,3.000000,80.000000,8.000000,9.000000,512.329200,1.000000,4.000000,11.000000,4.000000


In [49]:
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)

In [50]:
train_df['Sex'] = train_df['Sex'].astype('category')
train_df['Embarked'] = train_df['Embarked'].astype('category')
train_df['ageGroup'] = train_df['ageGroup'].astype('category')
train_df['FareGroup'] = train_df['FareGroup'].astype('category')
train_df['solo'] = train_df['solo'].astype('category')
train_df['Pclass'] = train_df['Pclass'].astype('category')
train_df['FamilySize'] = train_df['FamilySize'].astype('category')
train_df['SibSp'] = train_df['SibSp'].astype('category')
train_df['Parch'] = train_df['Parch'].astype('category')

In [51]:
X = train_df[:X_train.shape[0]]
y = y_train

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [57]:
def objective(trial):
    model = CatBoostClassifier(
        cat_features=['Sex', 'Embarked', 'ageGroup', 'FareGroup', 'solo', 'Pclass', 'FamilySize', 'SibSp', 'Parch'],
        iterations=trial.suggest_int("iterations", 500, 2000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 2, 4),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False,
        task_type='CPU'
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [58]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

bohb_sampler = optuna.samplers.TPESampler(n_startup_trials=30, n_ei_candidates=13, multivariate=True)
study = optuna.create_study(sampler=bohb_sampler, direction='maximize')
study.optimize(objective, n_trials=150, show_progress_bar=True, n_jobs=-1)

/opt/conda/lib/python3.11/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning:

``multivariate`` option is an experimental feature. The interface can change in the future.



  0%|          | 0/150 [00:00<?, ?it/s]

In [59]:
from optuna.visualization import plot_slice

plot_slice(study)

In [ ]:
# from catboost import CatBoostClassifier
# from sklearn.metrics import accuracy_score
# import numpy as np

# y = train_df['Survived']
# X = train_df.drop(columns=['Survived', 'Name', 'Ticket', 'Cabin'])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# classifier = CatBoostClassifier(
#     n_estimators=1000,
#     depth=2,
#     loss_function='Logloss',
#     verbose=False,
#     task_type='GPU',
#     devices='0:7',
#     cat_features=['Sex', 'Embarked', 'ageGroup', 'FareGroup', 'solo', 'Pclass', 'FamilySize', 'SibSp', 'Parch'],
#     eval_metric='Accuracy'
# )
# classifier.grid_search({'learning_rate': np.linspace(0.01, 0.1, 10),
#                         'l2_leaf_reg': np.linspace(0, 5, 10)},
#                        X_train, y_train, cv=5, plot=True, refit=True, verbose=False)

In [ ]:
# X_train = train_df[:X_train.shape[0]]
# X_test = train_df[X_train.shape[0]:]
# X_train

In [ ]:
# from catboost import CatBoostClassifier
# from sklearn.metrics import accuracy_score

# X_train = train_df[:X_train.shape[0]]
# X_test = train_df[X_train.shape[0]:]

# classifier = CatBoostClassifier(
#     n_estimators=1000,
#     depth=2,
#     learning_rate=0.01,
#     l2_leaf_reg=0.05263157895,
#     loss_function='Logloss',
#     verbose=False,
#     cat_features=['Sex', 'Embarked', 'ageGroup', 'FareGroup', 'solo', 'Pclass', 'FamilySize', 'SibSp', 'Parch'],
#     eval_metric='Accuracy',
# )

# X_train

In [ ]:
# classifier.fit(X_train, y_train)
# y_pred = classifier.predict(X_test)
# X_test['Survived'] = y_pred

In [ ]:
# X_test[['PassengerId', 'Survived']].to_csv('submission.csv', index=False)

In [ ]:
# ! kaggle competitions submit -c titanic -f submission.csv -m "first attemp"

In [ ]:
# попробуем применить случайный лес
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# используем one-hot encoding для категориальных признаков

from sklearn.preprocessing import OneHotEncoder

# X['Embarked'].fillna(X['Embarked'].mode()[0], inplace=True)

encoder = OneHotEncoder(sparse=False)
X_encoded = encoder.fit_transform(train_df[['Sex', 'Embarked']])
encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(['Sex', 'Embarked']))
train_df = train_df.drop(columns=['Sex', 'Embarked'])
train_df = pd.concat([train_df, encoded_df], axis=1)

X_train = train_df[:X_train.shape[0]]
X_test = train_df[X_train.shape[0]:]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

classifier = RandomForestClassifier(n_estimators=10000, max_depth=10, random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
# accuracy_score(y_test, y_pred)

/opt/conda/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



In [ ]:
X_test

,PassengerId,Pclass,Age,SibSp,Parch,Fare,solo,ageGroup,FamilySize,FareGroup,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
623,624,3,21.0,0,0,7.8542,0,2,1,1,0.0,1.0,0.0,0.0,1.0
624,625,3,21.0,0,0,16.1000,0,2,1,2,0.0,1.0,0.0,0.0,1.0
625,626,1,61.0,0,0,32.3208,0,4,1,4,0.0,1.0,0.0,0.0,1.0
626,627,2,57.0,0,0,12.3500,0,3,1,1,0.0,1.0,0.0,1.0,0.0
627,628,1,21.0,0,0,77.9583,0,2,1,4,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,3,28.0,0,0,8.0500,0,2,1,1,0.0,1.0,0.0,0.0,1.0
1305,1306,1,39.0,0,0,108.9000,0,2,1,4,1.0,0.0,1.0,0.0,0.0
1306,1307,3,38.5,0,0,7.2500,0,2,1,1,0.0,1.0,0.0,0.0,1.0
1307,1308,3,28.0,0,0,8.0500,0,2,1,1,0.0,1.0,0.0,0.0,1.0


In [ ]:
X_test['Survived'] = y_pred
X_test[['PassengerId', 'Survived']].to_csv('submission.csv', index=False)

/tmp/ipykernel_53313/3428381318.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# ! kaggle competitions submit -c titanic -f submission.csv -m "second attemp"

100%|██████████████████████████████████████| 4.34k/4.34k [00:00<00:00, 7.07kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

In [53]:
import pandas as pd
import seaborn as sns
import optuna
from optuna.samplers import TPESampler
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train = pd.read_csv('/opt/notebooks/kaggle/Kaggle-competitions/titanic/train.csv')
X_test = pd.read_csv('/opt/notebooks/kaggle/Kaggle-competitions/titanic/test.csv')

y_train = X_train['Survived']
X_train.drop(columns='Survived', inplace=True)

train_df = pd.concat([X_train, X_test], ignore_index=True)
train_df = train_df.drop(columns=['Ticket', 'Cabin'])
train_df['Fare'].fillna(train_df['Fare'].median(), inplace=True)

In [54]:
test_age = train_df[train_df['Age'].isna()]
train_age = train_df[train_df['Age'].notna()]
# обучаем модель на данных, где возраст не пропущен

X = train_age[['Pclass', 'SibSp', 'Parch', 'Fare']]
y = train_age['Age']

# X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(X, y, test_size=0.3, random_state=42)
model = RandomForestRegressor(n_estimators=1000, max_depth=10, random_state=42)
model.fit(X, y)
predicted_ages = model.predict(test_age[['Pclass', 'SibSp', 'Parch', 'Fare']]).astype(int)
train_df.loc[train_df['Age'].isna(), 'Age'] = predicted_ages

In [55]:
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)

In [56]:
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
train_df['IsAlone'] = 1
train_df['IsAlone'].loc[train_df['FamilySize'] > 1] = 0
train_df['FareBin'] = pd.qcut(train_df['Fare'], 4, labels=False)
train_df['AgeBin'] = pd.cut(train_df['Age'].astype(int), 5, labels=False)

/tmp/ipykernel_221872/167332781.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
train_df['Title'] = train_df['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
stat_min = 10
title_names = (train_df['Title'].value_counts() < stat_min)
train_df['Title'] = train_df['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)
print(train_df['Title'].value_counts())

Mr        757
Miss      260
Mrs       197
Master     61
Misc       34
Name: Title, dtype: int64


In [58]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1309 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Fare         1309 non-null   float64
 8   Embarked     1309 non-null   object 
 9   FamilySize   1309 non-null   int64  
 10  IsAlone      1309 non-null   int64  
 11  FareBin      1309 non-null   int64  
 12  AgeBin       1309 non-null   int64  
 13  Title        1309 non-null   object 
dtypes: float64(2), int64(8), object(4)
memory usage: 143.3+ KB


In [59]:
train_df.drop(columns=['Name'], inplace=True)

In [60]:
train_df['Sex'] = train_df['Sex'].astype('category')
train_df['Embarked'] = train_df['Embarked'].astype('category')
train_df['Title'] = train_df['Title'].astype('category')
train_df['IsAlone'] = train_df['IsAlone'].astype('category')
train_df['Pclass'] = train_df['Pclass'].astype('category')
train_df['FamilySize'] = train_df['FamilySize'].astype('category')
train_df['SibSp'] = train_df['SibSp'].astype('category')
train_df['Parch'] = train_df['Parch'].astype('category')

In [61]:
X = train_df[:X_train.shape[0]]
y = y_train

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [62]:
def objective(trial):
    model = CatBoostClassifier(
        cat_features=['Sex', 'Embarked', 'FareBin', 'AgeBin', 'Title', 'IsAlone', 'Pclass', 'FamilySize', 'SibSp', 'Parch'],
        iterations=trial.suggest_int("iterations", 500, 2000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 2, 4),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False,
        task_type='CPU'
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [63]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

bohb_sampler = optuna.samplers.TPESampler(n_startup_trials=70, n_ei_candidates=13, multivariate=True)
study = optuna.create_study(sampler=bohb_sampler, direction='maximize')
study.optimize(objective, n_trials=300, show_progress_bar=True, n_jobs=-1)

/opt/conda/lib/python3.11/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning:

``multivariate`` option is an experimental feature. The interface can change in the future.



  0%|          | 0/300 [00:00<?, ?it/s]

In [64]:
from optuna.visualization import plot_slice

plot_slice(study)

In [65]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [66]:
study.best_params

{'iterations': 723,
 'learning_rate': 0.08620134007004433,
 'depth': 4,
 'l2_leaf_reg': 0.1974080708861932,
 'bootstrap_type': 'Bayesian',
 'random_strength': 0.5693541230488852,
 'bagging_temperature': 1.6762236111624398,
 'od_type': 'Iter',
 'od_wait': 49}

In [68]:
X_test = train_df[X.shape[0]:]

model = CatBoostClassifier(**study.best_params, cat_features=['Sex', 'Embarked', 'FareBin', 'AgeBin', 'Title', 'IsAlone', 'Pclass', 'FamilySize', 'SibSp', 'Parch'], verbose=False)
model.fit(X, y)
pred = model.predict(X_test)

In [74]:
X_test['Survived'] = pred
X_test[['PassengerId', 'Survived']].to_csv('/opt/notebooks/kaggle/Kaggle-competitions/titanic/submission.csv', index=False)

/tmp/ipykernel_221872/1388606950.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [72]:
! kaggle competitions submit -c titanic -f /opt/notebooks/kaggle/Kaggle-competitions/titanic/submission.csv -m "catboost and optuna"

^C
User cancelled operation
